In [9]:
import os
import pathlib
import json

In [10]:
import pandas as pd 
import numpy as np

### enter bulletin directory 

In [11]:
bulletin_dir = 'JJA_2019'

In [12]:
dpath = pathlib.Path.cwd().parents[1] / 'bulletin' / bulletin_dir

In [13]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/JJA_2019')

### reads the country dictionnary for the TRMM rainfall time-series

In [14]:
with open(pathlib.Path.cwd().parents[1] / 'resources' / 'dict_countries.json', 'r') as fj: 
    dict_countries = json.load(fj)

### read the forecast table 

In [15]:
forecast = pd.read_csv(dpath / 'copernicus_forecasts_{}.csv'.format(bulletin_dir), index_col=0)

In [16]:
forecast.head()

,T1,T2,T3,country_fname,std,CONFIDENCE
country_name,,,,,,
Nauru,2,2,96,Nauru,26.095783,High
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate


In [29]:
def terciles_to_category(df, colnames=['T1','T2','T3'], mindiff=10, climdiff=5): 
    """
    convert a tercile probabilistic forecast into a categorical forecast
    
    if the difference between the maximum probability and the next one is >= mindiff 
    (10 is default) then it's either 'ABOVE', 'NEAR NORMAL' or 'BELOW' depending on the max 
    probability 
    
    if the difference between consecutive probabilities is less than mindiff (10)
    but greater than climdiff (5) then we end up with either 'AVG - BELOW' or 'AVG - ABOVE'
    
    if the difference between consecutive probabilities is less than 5%, it's a CLIMATOLOGY 
    forecasts 
    """
    
    T1, T2, T3 = colnames
    
    cats = []
    for i, row in df.iterrows(): 
        fcst = row.loc[colnames,].astype('int')
        if (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) >= mindiff): 
            cat = 'ABOVE'
#         elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T2]) > climdiff) & ((fcst.loc[T3] - fcst.loc[T1]) > mindiff):
        elif (fcst.idxmax()  == T3) & ((fcst.loc[T3] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T3] - fcst.loc[T1]) > mindiff):
            cat = 'AVG - ABOVE'
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) >= mindiff):
            cat = 'BELOW'
#         elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T2]) > climdiff) & ((fcst.loc[T1] - fcst.loc[T3]) > mindiff):
        elif (fcst.idxmax()  == T1) & ((fcst.loc[T1] - fcst.loc[T2]) < mindiff) & ((fcst.loc[T1] - fcst.loc[T3]) > mindiff):
            cat = 'AVG - BELOW'
        elif (fcst.idxmax()  == T2) & ((fcst.loc[T2] - fcst.loc[T1]) >= mindiff) & ((fcst.loc[T2] - fcst.loc[T3]) >= mindiff): 
            cat = 'NEAR NORMAL'
        else: 
            cat = 'CLIMATOLOGY'
        cats.append(cat)
    return np.array(cats)

In [30]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Nauru,2,2,96,Nauru,26.095783,High,ABOVE
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High,ABOVE
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High,ABOVE
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High,ABOVE
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate,ABOVE
Kiribati: Line Islands,25,32,43,Kiribati:_Line_Islands,14.624954,High,ABOVE
Austral Islands,26,32,42,Austral_Islands,11.493066,High,ABOVE
Tokelau,30,30,40,Tokelau,20.133254,Moderate,ABOVE
Pitcairn Islands,29,32,39,Pitcairn_Islands,13.405402,High,CLIMATOLOGY


In [31]:
forecast.loc[:,'CATEGORY'] = terciles_to_category(forecast)

In [33]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Nauru,2,2,96,Nauru,26.095783,High,ABOVE
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High,ABOVE
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High,ABOVE
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High,ABOVE
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate,ABOVE
Kiribati: Line Islands,25,32,43,Kiribati:_Line_Islands,14.624954,High,ABOVE
Austral Islands,26,32,42,Austral_Islands,11.493066,High,ABOVE
Tokelau,30,30,40,Tokelau,20.133254,Moderate,ABOVE
Pitcairn Islands,29,32,39,Pitcairn_Islands,13.405402,High,CLIMATOLOGY


### saves to the folder for inclusion in the PPT 

#### This is the table that appears on page 2 of the ICu bulletin

In [12]:
ppt_forecast_table = forecast.loc[:,['T1','T2','T3','CATEGORY','CONFIDENCE']]

In [13]:
ppt_forecast_table = ppt_forecast_table.sort_values(by=['T3','T2','T1'], ascending=False)

In [14]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Tuamotu / Gambier Islands':'Tuamotu Islands'})

In [15]:
ppt_forecast_table = ppt_forecast_table.rename(index={'Federated States of Micronesia':'FSM'})

In [16]:
ppt_forecast_table.to_csv(dpath / 'ICU_forecast_table_for_PPT_{}.csv'.format(bulletin_dir))

### code the categories for the water stress calculations

In [17]:
dict_replace = {}

dict_replace['ABOVE'] = 4

dict_replace['AVG - ABOVE'] = 2
dict_replace['AVG-ABOVE'] = 2
dict_replace['AVG – ABOVE'] = 2
dict_replace['AVG–ABOVE'] = 2

dict_replace['NEAR NORMAL'] = 0
dict_replace['NEAR NORMAL '] = 0
dict_replace['CLIMATOLOGY'] = 0


dict_replace['AVG - BELOW'] = -3
dict_replace['AVG-BELOW'] = -3
dict_replace['AVG – BELOW'] = -3
dict_replace['AVG–BELOW'] = -3

dict_replace['BELOW'] = -5

dict_replace['CLIMATOLOGY'] = -666

In [18]:
forecast = forecast.replace({'CATEGORY':dict_replace})

In [19]:
forecast

,T1,T2,T3,country_fname,std,CONFIDENCE,CATEGORY
country_name,,,,,,,
Nauru,2,2,96,Nauru,26.095783,High,4
Kiribati: Gilbert Islands,3,4,93,Kiribati:_Gilbert_Islands,19.975656,High,4
Federated States of Micronesia,16,20,64,Federated_States_of_Micronesia,11.410921,High,4
Kiribati: Phoenix Islands,18,23,59,Kiribati:_Phoenix_Islands,15.900944,Moderate-High,4
Tuvalu,21,25,54,Tuvalu,20.624085,Moderate,4
Kiribati: Line Islands,25,32,43,Kiribati:_Line_Islands,14.624954,High,4
Austral Islands,26,32,42,Austral_Islands,11.493066,High,4
Tokelau,30,30,40,Tokelau,20.133254,Moderate,4
Pitcairn Islands,29,32,39,Pitcairn_Islands,13.405402,High,2


### changes the names of the columns etc and saves the forecast table for mapping 

In [20]:
table_for_mapping = forecast.copy()

In [21]:
table_for_mapping.loc[:,'COUNTRYNAM'] = table_for_mapping.index

In [22]:
table_for_mapping.rename({'CATEGORY':'Stress'}, axis=1, inplace=True)

In [23]:
table_for_mapping = table_for_mapping.loc[:,['Stress','COUNTRYNAM']]

In [24]:
table_for_mapping.index.name = None

In [25]:
table_for_mapping.to_csv(dpath / 'ICU_forecast_table_for_mapping.csv')

### Now read the last 6 months rainfall categories from TRMM

In [126]:
last_6_months = pd.read_csv(dpath / 'last_6_months_TRMM_categories.csv', index_col=0)

In [127]:
last_6_months.set_index('month', inplace=True)

In [128]:
last_6_months

,Palau,Papua_New_Guinea,Society_Islands,Niue,Kiribati_Phoenix_Islands,Wallis_and_Futuna,Marquesas,Fiji,Kiribati_Line_Islands,Vanuatu_North,...,Tuvalu,Solomon_Islands,Tonga,Pitcairn_Islands,Guam,Northern_Marianas,Federated_States_of_Micronesia,Marshall_Islands,American_Samoa,year
month,,,,,,,,,,,,,,,,,,,,,
11,Normal,Normal,Below,Well below,Well above,Below,Normal,Normal,Above,Well below,...,Well above,Well above,Well below,Below,Above,Above,Below,Below,Below,2018
12,Normal,Above,Well below,Well above,Normal,Well above,Above,Well above,Normal,Well above,...,Normal,Well above,Well above,Below,Above,Below,Below,Above,Well above,2018
1,Below,Normal,Above,Above,Normal,Above,Normal,Well above,Above,Below,...,Below,Below,Well above,Well below,Normal,Below,Well above,Normal,Well below,2019
2,Well below,Well below,Normal,Below,Well above,Well above,Well below,Normal,Above,Above,...,Well above,Above,Well above,Above,Well above,Above,Below,Above,Well above,2019
3,Above,Well above,Normal,Well below,Above,Below,Below,Above,Above,Well below,...,Well below,Well below,Well above,Well above,Below,Well below,Above,Normal,Below,2019
4,Normal,Above,Above,Above,Well above,Well below,Normal,Well above,Normal,Above,...,Normal,Above,Normal,Below,Normal,Well below,Normal,Well below,Below,2019


In [129]:
last_6_months = last_6_months.T

In [130]:
dict_replace = {}
dict_replace['Well below'] = -5 
dict_replace['Below'] = -3 
dict_replace['Normal'] = 0
dict_replace['Above'] = 2
dict_replace['Well above'] = 4

In [131]:
last_6_months.head()

month,11,12,1,2,3,4
Palau,Normal,Normal,Below,Well below,Above,Normal
Papua_New_Guinea,Normal,Above,Normal,Well below,Well above,Above
Society_Islands,Below,Well below,Above,Normal,Normal,Above
Niue,Well below,Well above,Above,Below,Well below,Above
Kiribati_Phoenix_Islands,Well above,Normal,Normal,Well above,Above,Well above


In [132]:
last_6_months  = last_6_months.replace(dict_replace)

In [133]:
last_6_months = last_6_months.sort_index()

In [134]:
last_6_months.columns = range(-6,0)

In [135]:
last_6_months = last_6_months.rename(index=dict_countries)

In [136]:
last_6_months.head()

,-6,-5,-4,-3,-2,-1
American Samoa,-3,4,-5,4,-3,-3
Austral Islands,-3,0,4,4,0,0
Federated States of Micronesia,-3,-3,4,-3,2,0
Fiji,0,4,4,0,2,4
Guam,2,2,0,4,-3,0


In [137]:
last_6_months.index.name = 'country_name'

### now calculates the water stress itself 

In [138]:
water_stress = forecast.loc[:,['CATEGORY']]

In [139]:
water_stress = water_stress.merge(last_6_months, left_index=True, right_index=True)

### replace the code for climatology (-666) with 0 (i.e. equivalent to normal) for calculation of the water stress level

In [140]:
water_stress = water_stress.replace({'CATEGORY':{-666:0}})

In [141]:
water_stress.loc[:,'Stress'] = water_stress.loc[:,-1] +  water_stress.loc[:,-2] + \
0.8 * water_stress.loc[:,-3] + \
0.6 * water_stress.loc[:,-4] + \
0.4 * water_stress.loc[:,-5] + \
0.2 * water_stress.loc[:,-6] + \
water_stress.loc[:,'CATEGORY']

In [142]:
water_stress.head()

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress
country_name,,,,,,,,
Kiribati: Gilbert Islands,4,0,2,4,4,4,4,18.4
Nauru,4,4,0,2,4,4,4,17.2
Kiribati: Phoenix Islands,4,4,0,0,4,2,4,14.0
Kiribati: Line Islands,4,2,0,2,2,2,0,9.2
Tuvalu,4,4,0,-3,4,-5,0,1.2


### now determine the water stress catergory from the water stress value

In [143]:
bins = [-np.inf, -10, -7, -2, 2, 6, 11, np.inf]

In [144]:
labels = [-3, -2, -1, 0, 1, 2, 3]

In [145]:
water_stress.loc[:,'stress_category'] = pd.cut(water_stress.Stress.values.flatten(), bins=bins, labels=labels)

In [146]:
water_stress

,CATEGORY,-6,-5,-4,-3,-2,-1,Stress,stress_category
country_name,,,,,,,,,
Kiribati: Gilbert Islands,4,0,2,4,4,4,4,18.4,3
Nauru,4,4,0,2,4,4,4,17.2,3
Kiribati: Phoenix Islands,4,4,0,0,4,2,4,14.0,3
Kiribati: Line Islands,4,2,0,2,2,2,0,9.2,2
Tuvalu,4,4,0,-3,4,-5,0,1.2,0
Federated States of Micronesia,4,-3,-3,4,-3,2,0,4.2,1
Wallis & Futuna,4,-3,4,2,4,-3,-5,1.4,0
Tokelau,4,-3,-3,0,2,-5,-5,-6.2,-1
Samoa,4,-3,2,-3,4,-5,-5,-4.4,-1


In [147]:
water_stress.index.name = 'locationID'

In [148]:
dpath

PosixPath('/home/nicolasf/operational/ICU/ops/bulletin/MJJ_2019')

### now saves for mapping 

In [149]:
water_stress.to_csv(dpath / 'ICU_stress_table_for_mapping.csv')